### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# Files to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
schools_df = pd.merge(student_data, school_data, how="left", on=["school_name"])

## District Summary

This section creates the District Summary Report:
* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [ ]:
# PREPARE VALUES FOR THE DISTRICT SUMMARY REPORT
# Calculate the total number of schools
num_schools = len(schools_df['School ID'].unique())

# Calculate total number of students
num_students = len(schools_df['Student ID'].unique())

# Calculate the total budget
tot_budget = school_data['budget'].sum()

# Calculate the average math score
avg_math_score = schools_df['math_score'].mean()

# Calculate the average reading score
avg_reading_score = schools_df['reading_score'].mean()

# Calculate the percentage of students with a passing math score (70 or greater) 
pct_passing_math = (len(schools_df.loc[schools_df['math_score'] >= 70])/num_students)*100

# Calculate the percentage of students with a passing reading score (70 or greater)
pct_passing_reading = (len(schools_df.loc[schools_df['reading_score'] >= 70])/num_students)*100

# Calculate the percentage of students who passed math and reading (% Overall Passing)
pct_overall_passing = (len(schools_df.loc[(schools_df['reading_score'] >= 70) & (schools_df['math_score'] >= 70)])/num_students)*100


In [ ]:
# Creating a summary DataFrame for the District
district_summary_df = pd.DataFrame({'Total Schools': [num_schools], 
                                     'Total Students': [num_students],
                                     'Total Budget': [tot_budget],
                                     'Average Math Score': [avg_math_score],
                                     'Average Reading Score': [avg_reading_score],
                                     '% Passing Math': [pct_passing_math],
                                     '% Passing Reading': [pct_passing_reading],
                                     '% Overall Passing': [pct_overall_passing],
                                    })


In [ ]:
# format the table for better readability
district_summary_df['Total Students'] = district_summary_df['Total Students'].map("{:,}".format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map("${:,.2f}".format)
district_summary_df.head()
district_summary_df.style.hide_index()


In [ ]:
district_summary_df.dtypes

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [2]:
# PREPARE VALUES FOR THE SCHOOL SUMMARY REPORT
school_group = schools_df.groupby(["school_name"])

# Total students
student_counts = school_group["Student ID"].count()

# total school budget
total_school_budget = school_group['budget'].mean()

# per student budget
per_student_budget = school_group['budget'].mean()/school_group["Student ID"].count() 

# average math score
average_math_score = school_group["math_score"].mean()

# average reading score
average_reading_score = school_group["reading_score"].mean()

# percent passing math
number_passing_math = schools_df.groupby('school_name')['math_score'].apply(lambda x: x[x >= 70].count())
pct_passing_math = (number_passing_math/student_counts)*100

# percent passing reading
number_passing_reading = schools_df.groupby('school_name')['reading_score'].apply(lambda x: x[x >= 70].count())
pct_passing_reading = (number_passing_reading/student_counts)*100

# percent passing both
passing_both_df = schools_df.loc[(schools_df['math_score'] >= 70) & (schools_df['reading_score'] >= 70)]
number_passing_both = passing_both_df.groupby('school_name')['Student ID'].count()
pct_passing_both = (number_passing_both/student_counts)*100

school_group.count().head()

,Student ID,student_name,gender,grade,reading_score,math_score,School ID,type,size,budget
school_name,,,,,,,,,,
Bailey High School,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976
Cabrera High School,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858
Figueroa High School,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949
Ford High School,2739,2739,2739,2739,2739,2739,2739,2739,2739,2739
Griffin High School,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468


In [3]:
# Creating the School summary DataFrame
school_summary_df = pd.DataFrame({   'Total Students': student_counts,
                                     'Total School Budget': total_school_budget,
                                     'Per Student Budget': per_student_budget,
                                     'Average Math Score': average_math_score,
                                     'Average Reading Score': average_reading_score,
                                     '% Passing Math': pct_passing_math,
                                     '% Passing Reading': pct_passing_reading,
                                     '% Overall Passing': pct_passing_both
                                    })

# Clean up the School summary dataframe
merged_school_summary_df = pd.merge(school_summary_df,school_data[['school_name','type']],on='school_name', how='left')
merged_school_summary_df.head()
reorganized_school_summary_df = merged_school_summary_df[['school_name', 'type','Total Students', 'Total School Budget',
       'Per Student Budget', 'Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading', '% Overall Passing']]
final_school_summary_df = reorganized_school_summary_df.rename(columns={'school_name':'School Name', 'type':'School Type'})


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [5]:
# format the table for better readability
final_school_summary_df['Total School Budget'] = final_school_summary_df['Total School Budget'].map("${:,.2f}".format)
final_school_summary_df['Per Student Budget'] = final_school_summary_df['Per Student Budget'].map("${:,.2f}".format)
final_school_summary_df.head(20)
final_school_summary_df.style.hide_index()


School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [ ]:
# Sort and display the top five performing schools by % overall passing
top_schools_df = final_school_summary_df.sort_values("% Overall Passing", ascending=False)
top_schools_df.head(5)


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [ ]:
# Sort and display the five worst performing schools by % overall passing
bottom_schools_df = final_school_summary_df.sort_values("% Overall Passing")
bottom_schools_df.head(5)

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
# Create a series for each grade (9-12)
ninth_grade_df = schools_df.loc[schools_df['grade'] == '9th', :]
tenth_grade_df = schools_df.loc[schools_df['grade'] == '10th', :]
eleventh_grade_df = schools_df.loc[schools_df['grade'] == '11th', :]
twelfth_grade_df = schools_df.loc[schools_df['grade'] == '12th', :]

# Group each series by school and average the reading scor
ninth_grade_group = ninth_grade_df.groupby('school_name')
ninth_grade_average_math = ninth_grade_group['math_score'].mean()

tenth_grade_group = tenth_grade_df.groupby('school_name')
tenth_grade_average_math = tenth_grade_group['math_score'].mean()

eleventh_grade_group = eleventh_grade_df.groupby('school_name')
eleventh_grade_average_math = eleventh_grade_group['math_score'].mean()

twelfth_grade_group = twelfth_grade_df.groupby('school_name')
twelfth_grade_average_math = twelfth_grade_group['math_score'].mean()


In [ ]:
# Combine the series into a dataframe
math_scores_df = pd.DataFrame({       '9th': ninth_grade_average_math,
                                     '10th': tenth_grade_average_math,
                                     '11th': eleventh_grade_average_math,
                                     '12th': twelfth_grade_average_math
                                 })

In [ ]:
math_scores_df.head(20)

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
# average the math scores using school groups creating in previous section (math scores)
ninth_grade_average_reading = ninth_grade_group['reading_score'].mean()

tenth_grade_average_reading = tenth_grade_group['reading_score'].mean()

eleventh_grade_average_reading = eleventh_grade_group['reading_score'].mean()

twelfth_grade_average_reading = twelfth_grade_group['reading_score'].mean()


In [ ]:
# Combine the series into a dataframe
reading_scores_df = pd.DataFrame({   '9th': ninth_grade_average_reading,
                                     '10th': tenth_grade_average_reading,
                                     '11th': eleventh_grade_average_reading,
                                     '12th': twelfth_grade_average_reading
                                 })

In [ ]:
reading_scores_df.head(20)

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Find the best bins to use for this data
school_summary_df['Per Student Budget'].describe()

In [ ]:
# Create bins in which to place values based upon the statistical summary above
bins = [570, 590, 627, 640, 660]
        
# Create labels for these bins
group_labels = ["$570-590", "$591-627", "$628-640", "$641-660"]

# Slice the data and place it into the bins
pd.cut(school_summary_df["Per Student Budget"], bins, labels=group_labels)

# Place the data series into a new column inside of the DataFrame
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], bins, labels=group_labels)
school_summary_df.head(20)

In [ ]:
spending_group = school_summary_df.groupby('Spending Ranges (Per Student)')

# Find how many rows fall into each bin
print(spending_group['Per Student Budget'].count())

# Get the average of each column within the GroupBy object
spending_ranges_scores = spending_group[['Average Math Score', 'Average Reading Score', '% Passing Math','% Passing Reading', '% Overall Passing']].mean()
spending_ranges_scores.head()

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Find the best bins to use for this data
school_summary_df['Total Students'].describe()

In [ ]:
# Create bins in which to place values based upon the statistical summary above
bins = [400, 999, 1999, 5000]
        
# Create labels for these bins
group_labels = ["400-999", "1000-1999", "$2000-5000"]

# Slice the data and place it into the bins
pd.cut(school_summary_df["Total Students"], bins, labels=group_labels)

# Place the data series into a new column inside of the DataFrame
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], bins, labels=group_labels)
school_summary_df.head(20)

In [ ]:
size_group = school_summary_df.groupby('School Size')

# Find how many rows fall into each bin
print(spending_group['Per Student Budget'].count())

# Get the average of each column within the GroupBy object
school_size_scores = size_group[['Average Math Score', 'Average Reading Score', '% Passing Math','% Passing Reading', '% Overall Passing']].mean()
school_size_scores.head()

## Scores by School Type

* Perform the same operations as above, based on school type

In [ ]:
# merge the District back into the school_summary_df
complete_schools_df = pd.merge(school_summary_df, school_data[['school_name','type']], how='left', on=['school_name'])
# complete_schools_df.head()

# Transform the school type into categories so I can use it for binning the data (can't bin/cut on categorical data)
complete_schools_df["type"] = complete_schools_df["type"].astype('category')
complete_schools_df['type_cat'] = complete_schools_df['type'].cat.codes
# complete_schools_df.dtypes


In [ ]:
# Create School Type bins & labels 
bins = [-0.1, 0.9, 1.1]
group_labels = ["Charter", "District"]

# Slice the data and place it into the bins
pd.cut(complete_schools_df["type_cat"], bins, labels=group_labels)

# Place the data series into a new column inside of the DataFrame
complete_schools_df["School Type"] = pd.cut(complete_schools_df["type_cat"], bins, labels=group_labels)
# complete_schools_df.head(20)

In [ ]:
school_type_group = complete_schools_df.groupby('School Type')

# Find how many rows fall into each bin
print(school_type_group['Per Student Budget'].count())

# Get the average of each column within the GroupBy object
school_type_scores = school_type_group[['Average Math Score', 'Average Reading Score', '% Passing Math','% Passing Reading', '% Overall Passing']].mean()
school_type_scores.head()